+ 필요한 라이브러리 임포트
+ 프로젝트 전역 설정

In [77]:
import os
import json
import numpy as np
import pandas as pd
import joblib
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from pathlib import Path
from typing import Dict, Any, Tuple

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
import optuna

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use("Agg")
mpl.rcParams['font.family'] = 'Malgun Gothic'
mpl.rcParams['axes.unicode_minus'] = False
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

#### 프로그램 전체에 적용되는 **전역변수(Global Variables)** 정의
+ USE_OPTUNA = True/False    
    - 최적파라메터 탐색 기능 on/off
+ STACK_PASSTHROUGH = True/False    
    - True면 stacking 할때 oof prediction + X data 를 final estimator 에 전달        
    - False 면 oof prediction 만 전달
+ MODEL_ID = 1 ~ 4 의 정수
    - 훈련할 모델 선택

In [ ]:
THIS_YEAR = 2025
RANDOM_STATE = 42

# --- 사용자 지정 전역변수 ----
MODEL_ID = 1 
USE_OPTUNA = False
STACK_PASSTHROUGH = False
N_TRIALS = 40
# ------------------------------

# 결과물 파일 저장경로 설정
INPUT_DIR = Path("./input")
OUTPUT_DIR = Path("./output_model") / f"model_{MODEL_ID}"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
INPUT_DIR.mkdir(exist_ok=True, parents=True)

# 분석 데이터셋 컬럼 정의
COL_TIME = ['사용년월', '전체일수', '작업가능일수', '중복일수', '공휴일']
COL_WEATHER = ['최고기온', '최저기온', '강우', '풍속', '강설', '안개', '미세먼지']
COL_OPERATION = ['NIS', 'NOS', 'FIS', 'FOS', 'CIS', 'COS', 'NIGT', 'NOGT', 'FIGT', 'FOGT', 'CIGT', 'COGT']

#### 데이터 로드하는 함수 정의    
+ df = pd.read_csv(path_csv) : comma 로 분리된 csv 파일 불러옴
+ df = pd.read_csv(path_csv, sep='\t') : 탭(tab) 로 분리된 csv 파일 불러옴

In [ ]:
def load_and_prepare(path_csv: str = "data.csv") -> pd.DataFrame:
    """ CSV 로드 후 년, 월 파생변수 컬럼을 생성. 필요없는 컬럼 삭제 후 data2.csv 저장 """
    
    # csv 파일 로드 
    df = pd.read_csv(path_csv, sep='\t')    # csv 파일은 tab 으로 구분되어 있음

    # 전역변수에 정의된 데이터셋의 컬럼만 사용함
    df = df[COL_TIME + COL_WEATHER + COL_OPERATION]

    # datetime 형식 변환, year & month 생성    
    df['datetime'] = pd.to_datetime(df['사용년월'], errors='raise', format="%Y년%m월")  # '2025년03월' 문자열을format 을 지정하여 읽음
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    
    # 파생변수 생성
    df['weather_bad_ratio'] = df[COL_WEATHER].sum(axis=1) / df['전체일수']
    df['weather_bad_ratio'] = df['weather_bad_ratio'].round(4)
    
    # 분석에 필요없는 데이터 컬럼 삭제
    df.drop(columns=['사용년월', 'datetime'], inplace=True, errors='raise')
    
    # 파일로 저장(사용자 확인용)
    df.to_csv("data2.csv", index=False)

    return df

#### csv 파일 로드하고 데이터 상태 확인

+ year, month, weather_bad_ratio 컬럼이 정상 값으로 생성되었는지 확인한다.

In [80]:
df = load_and_prepare('data.csv')
df.head(5)  # 상위 5줄 데이터 확인

,전체일수,작업가능일수,중복일수,공휴일,최고기온,최저기온,강우,풍속,강설,안개,미세먼지,NIS,NOS,FIS,FOS,CIS,COS,NIGT,NOGT,FIGT,FOGT,CIGT,COGT,year,month,weather_bad_ratio
0,31,17,2,6,0,0,2,3,0,0,5,653,663,1592,1583,2038,2019,3171662,3281098,30727140,30522035,1977133,1902260,2010,1,0.322581
1,28,17,2,6,0,0,3,1,0,0,3,574,596,1421,1413,1746,1758,3188655,3219193,27322348,27752799,1673514,1646549,2010,2,0.250000
2,31,13,2,5,0,0,5,4,1,1,4,634,629,1595,1595,1975,1994,3636114,3623821,32201638,31851457,2017253,2022131,2010,3,0.483871
3,30,15,2,4,0,0,6,4,0,1,2,644,634,1619,1650,2189,2195,3096550,3039014,31953619,31968114,2092329,2105940,2010,4,0.433333
4,31,15,2,7,0,0,3,1,0,4,3,705,727,1709,1699,2256,2225,3861031,3724627,33669914,33807157,2269505,2227439,2010,5,0.354839


In [81]:
df.info()   # 데이터 값들의 type 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   전체일수               180 non-null    int64  
 1   작업가능일수             180 non-null    int64  
 2   중복일수               180 non-null    int64  
 3   공휴일                180 non-null    int64  
 4   최고기온               180 non-null    int64  
 5   최저기온               180 non-null    int64  
 6   강우                 180 non-null    int64  
 7   풍속                 180 non-null    int64  
 8   강설                 180 non-null    int64  
 9   안개                 180 non-null    int64  
 10  미세먼지               180 non-null    int64  
 11  NIS                180 non-null    int64  
 12  NOS                180 non-null    int64  
 13  FIS                180 non-null    int64  
 14  FOS                180 non-null    int64  
 15  CIS                180 non-null    int64  
 16  COS                180 non

#### weather_score 계산해주는 함수 정의

<font color=cyan>
+ weights 값 갯수는 COL_WEATHER 의 길이(현재 7)와 같아야 함.<br>
+ weights 값들의 합계는 1.0 이 되어야 함.<br>
+ weights 값들의 순서는 위에서 확인한 날씨 데이터 컬럼의 순서와 같아야 함(가중치 값들을 곱하는 순서).<br>
+ weights 가중치가 높으면 risk_score 증가 -> 작업일수 감소 이므로 weather_score 점수는 percentile 의 역순 (5,4,3,2,1) 로 부여<br>
</font>

### 주의
<font color=yellow>모델훈련이나 성능평가가 아니라 새로운 데이터로 predict 할 경우, percentile 을 저장해뒀다가 weather_score 생성을 별도로 해줘야 한다.
+ 새로운 예측은 create_weather_score_for_predict_new() 함수를 사용한다.</font>

In [ ]:
def create_weather_score(df: pd.DataFrame) -> Tuple[pd.DataFrame, np.ndarray]:
    data = df.copy()
    weather_features = [c for c in COL_WEATHER if c in data.columns]
    weather_data = data[weather_features].fillna(0).to_numpy(dtype=float)

    # ---- weights는 사용자가 수정해야함 ----
    weights = [0.15, 0.1, 0.3, 0.2, 0.05, 0.15, 0.05] # 2025-09-19 설정 저장
    
    # weather 데이터 컬럼들과 가중치의 곱
    risk_score = weather_data.dot(weights)

    percentiles = np.percentile(risk_score, [20, 40, 60, 80])

    def assign(score: float) -> int:
        if score <= percentiles[0]:
            return 5
        elif score <= percentiles[1]:
            return 4
        elif score <= percentiles[2]:
            return 3
        elif score <= percentiles[3]:
            return 2
        else:
            return 1

    # weather_score 컬럼에 날씨점수 저장
    data['weather_score'] = [assign(s) for s in risk_score]
    
    # percentiles 는 저장해두고 새로운 데이터 들어올때 사용해야 한다.
    return data, percentiles

def create_weather_score_for_predict_new(df: pd.DataFrame, percentiles: np.ndarray) -> pd.DataFrame:
    data = df.copy()
    weather_features = [c for c in COL_WEATHER if c in data.columns]
    weather_data = data[weather_features].fillna(0).to_numpy(dtype=float)

    # ---- weights는 사용자가 수정해야함 ----
    weights = [0.15, 0.1, 0.3, 0.2, 0.05, 0.15, 0.05] # 2025-09-19 설정 저장
    
    # weather 데이터 컬럼들과 가중치의 곱
    risk_score = weather_data.dot(weights)

    # percentiles 훈련시 값을 가져와서 사용
    def assign(score: float) -> int:
        if score <= percentiles[0]:
            return 5
        elif score <= percentiles[1]:
            return 4
        elif score <= percentiles[2]:
            return 3
        elif score <= percentiles[3]:
            return 2
        else:
            return 1

    # weather_score 컬럼에 날씨점수 저장
    data['weather_score'] = [assign(s) for s in risk_score]
    
    return data

#### weather_score 컬럼 생성하고 값 확인

+ df['weather_score'] 값을 확인.
+ weather_score 를 결정한 기준치인 weather_percentiles 값을 확인.

In [83]:
df, weather_percentiles = create_weather_score(df)
print(df.head(5))
print(weather_percentiles)

   전체일수  작업가능일수  중복일수  공휴일  최고기온  최저기온  강우  풍속  강설  안개  미세먼지  NIS  NOS   FIS   FOS   CIS   COS     NIGT     NOGT      FIGT      FOGT     CIGT     COGT  year  month  weather_bad_ratio  weather_score
0    31      17     2    6     0     0   2   3   0   0     5  653  663  1592  1583  2038  2019  3171662  3281098  30727140  30522035  1977133  1902260  2010      1           0.322581              2
1    28      17     2    6     0     0   3   1   0   0     3  574  596  1421  1413  1746  1758  3188655  3219193  27322348  27752799  1673514  1646549  2010      2           0.250000              4
2    31      13     2    5     0     0   5   4   1   1     4  634  629  1595  1595  1975  1994  3636114  3623821  32201638  31851457  2017253  2022131  2010      3           0.483871              1
3    30      15     2    4     0     0   6   4   0   1     2  644  634  1619  1650  2189  2195  3096550  3039014  31953619  31968114  2092329  2105940  2010      4           0.433333              1
4    31   

#### data age score 컬럼을 생성하는 함수 정의

+ bins = 3 : 데이터를 3개 구간으로 구분.
+ 2010~2014 : 1점, 나머지 2015~2024는 5점.

### 주의
<font color=yellow>모델훈련이나 성능평가가 아니라 새로운 데이터로 predict 할 경우, df['data_age_score'] = 5.0 으로 모두 fill 해야 한다.</font>

In [84]:
def create_data_age(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()
    age = THIS_YEAR - data['year'].astype(int)
    
    # 2010 ~ 2015, 2016 ~ 2020, 2021 ~ 2024
    percentiles = np.percentile(age, [33, 66])

    def assign(score: float) -> int:
        if score <= percentiles[0]:
            return 5
        elif score <= percentiles[1]:
            return 5
        else:
            return 1

    # data_age 컬럼에 점수 저장
    data['data_age_score'] = [assign(s) for s in age]

    return data

#### data_age_score 컬럼 생성하고 값 확인

+ df['data_age_score'] 값을 확인.

In [85]:
df = create_data_age(df)
df.head(5)

,전체일수,작업가능일수,중복일수,공휴일,최고기온,최저기온,강우,풍속,강설,안개,미세먼지,NIS,NOS,FIS,FOS,CIS,COS,NIGT,NOGT,FIGT,FOGT,CIGT,COGT,year,month,weather_bad_ratio,weather_score,data_age_score
0,31,17,2,6,0,0,2,3,0,0,5,653,663,1592,1583,2038,2019,3171662,3281098,30727140,30522035,1977133,1902260,2010,1,0.322581,2,1
1,28,17,2,6,0,0,3,1,0,0,3,574,596,1421,1413,1746,1758,3188655,3219193,27322348,27752799,1673514,1646549,2010,2,0.250000,4,1
2,31,13,2,5,0,0,5,4,1,1,4,634,629,1595,1595,1975,1994,3636114,3623821,32201638,31851457,2017253,2022131,2010,3,0.483871,1,1
3,30,15,2,4,0,0,6,4,0,1,2,644,634,1619,1650,2189,2195,3096550,3039014,31953619,31968114,2092329,2105940,2010,4,0.433333,1,1
4,31,15,2,7,0,0,3,1,0,4,3,705,727,1709,1699,2256,2225,3861031,3724627,33669914,33807157,2269505,2227439,2010,5,0.354839,3,1


In [86]:
# year 컬럼 삭제
df.drop(columns='year', inplace=True, errors='raise')
# 테이블 상태 확인
print(df.head(5))
print(df.tail(5))
# 파생변수를 포함한 데이터셋 저장 -> 사용자가 파생변수값을 확인할 수 있다.
df.to_csv('data3.csv')

   전체일수  작업가능일수  중복일수  공휴일  최고기온  최저기온  강우  풍속  강설  안개  미세먼지  NIS  NOS   FIS   FOS   CIS   COS     NIGT     NOGT      FIGT      FOGT     CIGT     COGT  month  weather_bad_ratio  weather_score  \
0    31      17     2    6     0     0   2   3   0   0     5  653  663  1592  1583  2038  2019  3171662  3281098  30727140  30522035  1977133  1902260      1           0.322581              2   
1    28      17     2    6     0     0   3   1   0   0     3  574  596  1421  1413  1746  1758  3188655  3219193  27322348  27752799  1673514  1646549      2           0.250000              4   
2    31      13     2    5     0     0   5   4   1   1     4  634  629  1595  1595  1975  1994  3636114  3623821  32201638  31851457  2017253  2022131      3           0.483871              1   
3    30      15     2    4     0     0   6   4   0   1     2  644  634  1619  1650  2189  2195  3096550  3039014  31953619  31968114  2092329  2105940      4           0.433333              1   
4    31      15     2    

In [87]:
# X, y 저장
y = df.pop("작업가능일수").values
X = df

#### preprocessor, model, stacking 관련 함수 정의

In [88]:
def build_preprocessor(X: pd.DataFrame) -> ColumnTransformer:
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
    
    scale_cols = [c for c in COL_OPERATION if c in X.columns]  # operation 데이터는 모두 StandardScaler 로 scaling

    transformers = []
    if scale_cols:
        transformers.append(("scaled_numeric",
                             Pipeline([("imputer", SimpleImputer(strategy="median")),
                                       ("scaler", StandardScaler())]),
                             scale_cols))
    # 명목형 변수가 없으므로 이것은 사실 사용되지 않음
    if cat_cols:
        transformers.append(("categorical",
                             Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                                       ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))]),
                             cat_cols))

    return ColumnTransformer(transformers=transformers, remainder="passthrough")

def make_models(model_id: int):
    if model_id == 1:
        base = RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=100)
    elif model_id == 2:
        base = XGBRegressor(random_state=RANDOM_STATE, n_estimators=100, max_depth=6, learning_rate=0.1, subsample=0.9)
    elif model_id == 3:
        base = GradientBoostingRegressor(random_state=RANDOM_STATE)
    elif model_id == 4:
        base = HistGradientBoostingRegressor(random_state=RANDOM_STATE)
    else:
        raise ValueError("MODEL_ID 는 1~4")
    meta = Ridge(alpha=1.0)
    return base, meta

def make_stacking(model_id: int, passthrough: bool):
    base, meta = make_models(model_id)
    estimators = [('base', base)]
    stack = StackingRegressor(estimators=estimators, final_estimator=meta, passthrough=passthrough)
    return stack

#### MODEL_ID 별 모델의 최적 파라메터 탐색 공간의 범위 정의

+ optuna 탐색기가 사용하는 값이다.  사용자가 수정할 수 있지만, 딱히 건드릴 이유는 없다.

In [ ]:
def get_param_space(model_id: int) -> Dict[str, Any]:
    if model_id == 1:
        return {
            "base__n_estimators": (100, 400),
            "base__max_depth": (3, 20),
            "base__min_samples_split": (2, 20),
            "base__min_samples_leaf": (1, 10),
            "final_estimator__alpha": (0.01, 100)
        }
    elif model_id == 2:
        return {
            "base__n_estimators": (100, 400),
            "base__max_depth": (3, 12),
            "base__learning_rate": (1e-3, 0.3),
            "base__subsample": (0.6, 1.0),
            "base__colsample_bytree": (0.6, 1.0),
            "final_estimator__alpha": (0.01, 100)
        }
    elif model_id == 3:
        return {
            "base__n_estimators": (100, 400),
            "base__learning_rate": (1e-3, 0.3),
            "base__max_depth": (2, 6),
            "base__subsample": (0.6, 1.0),
            "final_estimator__alpha": (0.01, 100)
        }
    elif model_id == 4:
        return {
            "base__learning_rate": (1e-3, 0.5),
            "base__max_depth": (2, 16),
            "base__max_bins": (64, 256),
            "base__l2_regularization": (0.0, 2.0),
            "final_estimator__alpha": (0.01, 100)
        }
    else:
        raise ValueError("MODEL_ID 는 1~4")

In [90]:
# optuna 가 사용하는 함수
def suggest_params(trial: optuna.trial.Trial, space: Dict[str, Tuple[float, float]]) -> Dict[str, Any]:
    params = {}
    for k, v in space.items():
        low, high = v
        if "learning_rate" in k or "l2_regularization" in k or "subsample" in k or "colsample_bytree" in k:
            params[k] = trial.suggest_float(k, low, high, log=True)
        elif "max_depth" in k or "max_bins" in k or "n_estimators" in k or "min_samples" in k:
            params[k] = trial.suggest_int(k, int(low), int(high))
        else:
            params[k] = trial.suggest_float(k, low, high)
    return params

In [91]:
# 예측결과 점수 계산 함수
def evaluate(y_true, y_pred) -> Dict[str, float]:
    return {
        "r2": float(r2_score(y_true, y_pred)),
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
    }

#### ----------- 모델링 MAIN -----------

+ 여기서 부터 모델링이 시작된다.

In [92]:
# preprocessor, model 을 결합한 pipeline 정의
pre = build_preprocessor(X)
model = make_stacking(MODEL_ID, STACK_PASSTHROUGH)
pipe = Pipeline([("pre", pre), ("model", model)])

In [93]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
kf = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

<font color=magenta>**### USE_OPTUNA = False 인 경우 아래부터 실행한다.**</font>

In [159]:
best_params = {}
if USE_OPTUNA:      # optuna 로 최적 파라메터 탐색
    study = optuna.create_study(direction="maximize")   # R2 score 최대화
    space = get_param_space(MODEL_ID)

    def objective(trial):
        params = suggest_params(trial, space)
        pipe.set_params(**{f"model__{k}": v for k, v in params.items()})
        scores = cross_val_score(pipe, X_train, y_train, scoring="r2", cv=kf, n_jobs=None)
        return float(np.mean(scores))

    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=False)
    best_params = study.best_params     # 최적 훈련파라메터 저장
    pipe.set_params(**{f"model__{k}": v for k, v in best_params.items()})   # 최적파라메터를 pipe 에 입력        
    with open(OUTPUT_DIR / f"bestparam_model_{MODEL_ID}.json", "w", encoding="utf-8") as f:
        json.dump(best_params, f, ensure_ascii=False, indent=2)     # 최적 파라메터를 json 파일에 저장(/input 폴더에 복사해놓고 수정한다음 USE_OPTUNA=False 로 수동훈련 가능)

    # optuna 최적 파라메터 탐색 history plot
    try:
        fig = optuna.visualization.plot_optimization_history(study)
        # Many environments require kaleido for write_image. Fallback handled below.
        fig.write_image(str(OUTPUT_DIR / f"optuna_history_model_{MODEL_ID}.png"))
    except Exception:
        hist = [t.value for t in study.trials if t.value is not None]
        plt.figure(figsize=(6, 3))
        plt.plot(hist)
        plt.xlabel("trial")
        plt.ylabel("r2")
        plt.tight_layout()
        plt.savefig(OUTPUT_DIR / f"optuna_history_model_{MODEL_ID}.png", dpi=160)
        plt.close()

else:       # input 폴더의 사용자 파라메터를 읽는다.
    param_path = INPUT_DIR / f"bestparam_model_{MODEL_ID}.json"
    if not param_path.exists():
        raise FileNotFoundError(f"{param_path} 가 존재하지 않습니다. USE_OPTUNA=True 로 먼저 실행해 저장하세요.")
    with open(param_path, "r", encoding="utf-8") as f:
        best_params = json.load(f)
    pipe.set_params(**{f"model__{k}": v for k, v in best_params.items()})

In [160]:
# 최종 훈련된 파라메터와 전체 훈련데이터로 model fit 해서 완성.
pipe.fit(X_train, y_train)

,steps,"[('pre', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scaled_numeric', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [161]:
# Train / Test 결과 출력
y_pred_tr = pipe.predict(X_train)
y_pred_te = pipe.predict(X_test)
metrics_train = evaluate(y_train, y_pred_tr)
metrics_test = evaluate(y_test, y_pred_te)
print("[TRAIN]", metrics_train)
print("[TEST ]", metrics_test)

[TRAIN] {'r2': 0.9752863736080956, 'rmse': 0.6556774618313312, 'mae': 0.4809562708263005}
[TEST ] {'r2': 0.9110764624483523, 'rmse': 1.174600829586208, 'mae': 0.9156243040491652}


In [162]:
# 훈련된 모델, meat 정보 저장(MODEL_ID별로 파일명 저장한다)
joblib.dump(pipe, OUTPUT_DIR / f"stacking_model_{MODEL_ID}.joblib")
print(f"모델 저장 완료: {OUTPUT_DIR / f'stacking_model_{MODEL_ID}.joblib'}")

# 모델 훈련 meta 정보 기록
meta = {
    "MODEL_ID": MODEL_ID,
    "USE_OPTUNA": USE_OPTUNA,
    "STACK_PASSTHROUGH": STACK_PASSTHROUGH,
    "best_params": best_params
}
with open(OUTPUT_DIR / "run_meta.json", "w", encoding="utf-8-sig") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)    # run_meta.json 은 방금 실행한 run 관련 정보를 담고 있음. 매 실행시 덮어쓰기됨.ㄴ

모델 저장 완료: output_model\model_1\stacking_model_1.joblib


In [163]:
# 모델 훈련, 성능평가결과를 plotting 하는 코드를 작성한다.

#### ----------- 새로운 데이터로 예측 -----------
+ 훈련된 모델을 사용하여 새로운 데이터에 대한 예측 실행.
+ 주의 : data2.csv 파일은 덮어쓰기 된다.

In [164]:
# 모델 파일 불러오기
model_loaded = joblib.load(OUTPUT_DIR / f"stacking_model_{MODEL_ID}.joblib")

In [165]:
df = load_and_prepare('data_new.csv')
df.head(5)  # 상위 5줄 데이터 확인

,전체일수,작업가능일수,중복일수,공휴일,최고기온,최저기온,강우,풍속,강설,안개,미세먼지,NIS,NOS,FIS,FOS,CIS,COS,NIGT,NOGT,FIGT,FOGT,CIGT,COGT,year,month,weather_bad_ratio
0,31,21,1,8,0,0,1,1,0,0,0,455,476,1452,1446,1806,1796,4508704,4449360,48627580,48029936,1717625,1716871,2025,1,0.064516
1,28,21,1,4,0,0,2,2,0,0,0,440,442,1314,1312,1597,1607,4397794,4269339,42824452,42403651,1500251,1500618,2025,2,0.142857
2,31,19,2,6,0,0,3,4,0,1,0,482,478,1478,1498,1879,1902,4581762,4309539,49831655,49505583,1757272,1759925,2025,3,0.258065
3,30,21,1,4,0,0,3,3,0,0,0,462,462,1490,1478,1792,1802,4694293,4513389,47963527,47554794,1707988,1730572,2025,4,0.200000
4,31,17,2,7,0,0,3,3,0,2,0,468,476,1508,1507,1852,1852,4798390,4640800,48737407,48292248,1786942,1762138,2025,5,0.258065


In [166]:
df.info()   # 컬럼 데이터 type 형식 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   전체일수               10 non-null     int64  
 1   작업가능일수             10 non-null     int64  
 2   중복일수               10 non-null     int64  
 3   공휴일                10 non-null     int64  
 4   최고기온               10 non-null     int64  
 5   최저기온               10 non-null     int64  
 6   강우                 10 non-null     int64  
 7   풍속                 10 non-null     int64  
 8   강설                 10 non-null     int64  
 9   안개                 10 non-null     int64  
 10  미세먼지               10 non-null     int64  
 11  NIS                10 non-null     int64  
 12  NOS                10 non-null     int64  
 13  FIS                10 non-null     int64  
 14  FOS                10 non-null     int64  
 15  CIS                10 non-null     int64  
 16  COS                10 non-nul

In [167]:
# weather_score, data_age_score 파생변수 컬럼 생성
df = create_weather_score_for_predict_new(df, weather_percentiles)
df['data_age_score'] = 5.0  # 최신데이터라면 모두 5점 준다.
df.head(5)

,전체일수,작업가능일수,중복일수,공휴일,최고기온,최저기온,강우,풍속,강설,안개,미세먼지,NIS,NOS,FIS,FOS,CIS,COS,NIGT,NOGT,FIGT,FOGT,CIGT,COGT,year,month,weather_bad_ratio,weather_score,data_age_score
0,31,21,1,8,0,0,1,1,0,0,0,455,476,1452,1446,1806,1796,4508704,4449360,48627580,48029936,1717625,1716871,2025,1,0.064516,5,5.0
1,28,21,1,4,0,0,2,2,0,0,0,440,442,1314,1312,1597,1607,4397794,4269339,42824452,42403651,1500251,1500618,2025,2,0.142857,4,5.0
2,31,19,2,6,0,0,3,4,0,1,0,482,478,1478,1498,1879,1902,4581762,4309539,49831655,49505583,1757272,1759925,2025,3,0.258065,2,5.0
3,30,21,1,4,0,0,3,3,0,0,0,462,462,1490,1478,1792,1802,4694293,4513389,47963527,47554794,1707988,1730572,2025,4,0.200000,3,5.0
4,31,17,2,7,0,0,3,3,0,2,0,468,476,1508,1507,1852,1852,4798390,4640800,48737407,48292248,1786942,1762138,2025,5,0.258065,2,5.0


In [168]:
# year 컬럼 삭제
df.drop(columns='year', inplace=True, errors='raise')
# 테이블 상태 확인
print(df.head(5))
# 파생변수를 포함한 데이터셋 저장 -> 사용자가 파생변수값을 확인할 수 있다.
df.to_csv('data3_new.csv')

# X, y 저장
y_new = df.pop("작업가능일수").values
X_new = df

   전체일수  작업가능일수  중복일수  공휴일  최고기온  최저기온  강우  풍속  강설  안개  미세먼지  NIS  NOS   FIS   FOS   CIS   COS     NIGT     NOGT      FIGT      FOGT     CIGT     COGT  month  weather_bad_ratio  weather_score  \
0    31      21     1    8     0     0   1   1   0   0     0  455  476  1452  1446  1806  1796  4508704  4449360  48627580  48029936  1717625  1716871      1           0.064516              5   
1    28      21     1    4     0     0   2   2   0   0     0  440  442  1314  1312  1597  1607  4397794  4269339  42824452  42403651  1500251  1500618      2           0.142857              4   
2    31      19     2    6     0     0   3   4   0   1     0  482  478  1478  1498  1879  1902  4581762  4309539  49831655  49505583  1757272  1759925      3           0.258065              2   
3    30      21     1    4     0     0   3   3   0   0     0  462  462  1490  1478  1792  1802  4694293  4513389  47963527  47554794  1707988  1730572      4           0.200000              3   
4    31      17     2    

In [169]:
# 새로운 데이터로 predict 한 결과를 출력
y_pred = model_loaded.predict(X_new)
metrics_predict = evaluate(y_new, y_pred)
print("[PREDICT ]", metrics_predict)

[PREDICT ] {'r2': 0.8935838627818649, 'rmse': 1.2396164815901214, 'mae': 1.0590487647157867}


In [170]:
# 모델 예측결과를 plotting 하는 코드를 작성한다. 